<a href="https://colab.research.google.com/github/AndriiGoz/traffic_signs_classification/blob/master/traffic_signs_classification_lenet5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classification of Traffic Signs with LeNet-5 CNN
The purpose of this project is to train an implementation of the LeNet-5 Convolutional Neural Network for the classification of traffic signs. The model will be used in an application, where the user can upload a photo of a traffic sign and get the prediction of its class.


### 1. Mount Google Drive
The dataset was taken from *The German Traffic Sign Recognition Benchmark (GTSRB): https://benchmark.ini.rub.de/gtsrb_dataset.html.* It consists of about 40.000 real colorful photos of german traffic signs. The images have a .ppm extension and their size varies from 15x15 to 250x250 pixels. We save the dataset on Google Drive and access it using *drive.mount* command.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


### 2. Import libraries
Here we just import libraries we need in our Project: 
- some standard ones as NumPy, OS, and Matplotlib;
- cv2, a powerful library developed for solving computer vision tasks (https://opencv.org/);
- [sklearn.model_selection.train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) for splitting the dataset into train and test subsets;
- some components from [tf.keras.models](https://www.tensorflow.org/api_docs/python/tf/keras/models)...;
- and from [tf.keras.layers](https://www.tensorflow.org/api_docs/python/tf/keras/layers) for building the model.

In [ ]:
import numpy as np
import random
import os
import cv2 as cv
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential, load_model
from keras.layers import Conv2D, Dense, Flatten, Rescaling, AveragePooling2D, Dropout

### 3. Read and pre-process image files
We start with reading images from the dataset. The images are distributed over 43 folders representing 43 classes. We loop through folders and through images, open them, resize to 32x32 pixels, convert from RGB to gray, and save them as np.arrays.

In [ ]:
images = []
labels = []
classes = 43

current_path = '/content/gdrive/My Drive/GTSRB/Final_Training/Images/'
#current_path = '/My Drive/GTSRB/Final_Training/Images/'

for i in range(classes):
    path = os.path.join(current_path, str(str(i).zfill(5)))
    img_folder = os.listdir(path)
    for j in img_folder:
        try:
            image = cv.imread(str(path+'/'+j))
            image = cv.resize(image, (32, 32))
            image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
            image = np.array(image)
            images.append(image)
            label = np.zeros(classes)
            label[i] = 1.0
            labels.append(label)
        except:
            pass

We divide images by 255 to get pixel values between 0.0 and 1.0. Finally, we have got a total amount of 39.209 images assigned to 43 classes.

In [ ]:
images = np.array(images)
images = images/255
labels = np.array(labels)
print('Images shape:', images.shape)
print('Labels shape:', labels.shape)

Images shape: (0,)
Labels shape: (0,)


### 4. Split Dataset into train and test subsets
The dataset has to be split now into train and test subsets. For the test subset, we take out a standard 20% of the dataset.

In [ ]:
X = images.astype(np.float32)
y = labels.astype(np.float32)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

print('X_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print('X_test shape:', X_test.shape)
print('y_test shape:', y_test.shape)

ValueError: ignored


Let's take a look at some samples from our dataset. We pick up 25 random images and plot them together with their labels.

In [ ]:
plt.figure(figsize=(12, 12))
start_index = 0
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    
    label = np.argmax(y_train[start_index+i])
    
    plt.xlabel('i={}, label={}'.format(start_index+i, label))
    plt.imshow(X_train[start_index+i], cmap='gray')
plt.show()

### 5. Build the model
For our classification task, we will use an implementation of LeNet-5 Convolutional Neural Network. LeNet-5 was designed by Yann LeCun and others in 1998 and was one of the earliest convolutional neural networks. Its architecture is extremely simple but very efficient.

There are three Convolutional Layers based on 5x5 filters and followed by average pooling with 2×2 patches. We use the ReLU function for activation as it leads to faster training. Then we add Dropout Layer with a factor of 0.2 to overcome overfitting. It means that 20% of the input will be randomly nullified to prevent strong dependencies between layers. We end up with Flattening and two Dense Layers. In the last Dense Layer, we have to assign the number of neurons equal to the number of classes, and the Softmax activation function to get probabilities between 0 and 1. The resulting number of weights in this network is 70,415.

In [ ]:
# Building the model
model = Sequential([
    Rescaling(1, input_shape=(32, 32, 1)),
    Conv2D(filters=6, kernel_size=(5, 5), activation='relu'),
    AveragePooling2D(pool_size=(2, 2)),
    Conv2D(filters=16, kernel_size=(5, 5), activation='relu'),
    AveragePooling2D(pool_size=(2, 2)),
    Conv2D(filters=120, kernel_size=(5, 5), activation='relu'),
    Dropout(0.2),
    Flatten(),
    Dense(units=120, activation='relu'),
    Dense(units=43, activation='softmax')
])

# Compilation of the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Model architecture
model.summary()

### 6. Train the model

In [ ]:
history = model.fit(X_train, y_train, epochs=50,
                    validation_data=(X_test, y_test))

### 7. Evaluate training results
After 50 epochs we received an accuracy of about 99%, which is quite good. Thus we stop here.

In [ ]:
val_loss, val_acc = model.evaluate(X_test, y_test, verbose=2)
print('\nValifdation accuracy:', val_acc)
print('\nValidation loss:', val_loss)


Let's create plots of loss and accuracy on the training and validation sets.

In [ ]:
plt.figure(0)
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.8, 1])
plt.legend(loc='lower right')

plt.figure(1)
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim([0, 0.2])
plt.legend(loc='lower right')

As we see from the plots, train and validation accuracy go close, validation loss doesn't go up. So, the model looks fine, and we do not have much overfitting here.

### 8. Prediction for sampless
Let's take a look at some samples and find the wrong classified pictures. We label the images with *prediction* and *ground truth* classes. If prediction equals ground truth, we assign a green color to the label, otherwise we make it red.

In [ ]:
preds = model.predict(X_test)

plt.figure(figsize=(12, 12))
start_index = random.randint(0, 7800)
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    
    pred = np.argmax(preds[start_index+i])
    gt = np.argmax(y_test[start_index+i])
    
    col = 'g'
    if pred != gt:
        col = 'r'
    
    plt.xlabel('i={}, pred={}, gt={}'.format(start_index+i, pred, gt), color=col)
    plt.imshow(X_test[start_index+i], cmap='gray')
plt.show()

For image number 5955 we can see that the "Speed limit (50km/h)" sign was misclassified as "Speed limit (80km/h)". Obviously, the text on the sign here is very hard to recognize.

### 9. Save the model
In the end, we save the model to a separate folder on Google Drive. It will be used for further predictions in the application for traffic signs recognition.

In [ ]:
model.save('/content/gdrive/My Drive/keras_model/')